In [1]:
import numpy as np
import numpy as np
from concrete import fhe
from Bio.Seq import Seq, MutableSeq

import sys, os
sys.path.append(os.path.dirname(os.getcwd()))

from concrete_biopython.FheSeq import FheSeq, FheMutableSeq
from concrete_biopython.SeqWrapper import SeqWrapper


In [17]:
seq1 = Seq('AA')
seq2 = Seq('AAA')
seq3 = Seq('AAB')
seq4 = Seq('CAA')
seq = seq1+seq2+seq3+seq4

In [25]:
import functools

def sort_rank(seqlist, lib=fhe):
    n=len(seqlist)
    # create comparison matrix
    comp_matrix = lib.zeros((n,n))
    # fill the matrix with results of seqlist[i]>seqlist[j]
    for i in range(0, n):
        for j in range(i+1, n):
            # compute upper half of the matrix
            comp_matrix[i,j] = 1*(seqlist[i]<seqlist[j])
            # then copy the opposite to the lower half because the matrix is antisymetrical
            comp_matrix[j,i] = 1-comp_matrix[i,j] # this counts seqlist[j]>=seqlist[i]
    # now sum up each row to get a sorting rank for the sequences
    return np.sum(comp_matrix, axis=0)
            
def process_seq(s, lib=fhe):
    s1=s[0:2]
    s2=s[2:5]
    s3=s[5:8]
    s4=s[8:]
    to_sort = [s3,s4,s1,s2]
    return sort_rank(to_sort, lib)

In [26]:
output_seq = process_seq(seq, np)
print(output_seq)

[2. 3. 0. 1.]


In [27]:
# wrap a fhe circuit in order to input and output Bio.Seq objects.
def circuit_wrapper(circuit, seq, simulate=False):
    # convert Seq objects to integers with SeqWrapper.toIntegers
    integers = SeqWrapper(seq).toIntegers()
    # run the circuit with integer inputs
    integer_output = circuit.simulate(integers) if simulate else circuit.encrypt_run_decrypt(integers)
    # convert back the integer outputs into a Seq objects with SeqWrapper.toSeq
    return integer_output

In [30]:
def process_seq_adapter(integer_seq):
    # convert integer to FheSeq object, process it and return result
    return process_seq(FheSeq(integer_seq))

In [31]:
# compile the process_seq_adapter function and create a circuit
compiler = fhe.Compiler(lambda data: process_seq_adapter(data), {"data": "encrypted"})
circuit = compiler.compile(
    inputset=[
    np.random.randint(0, SeqWrapper.maxInteger()+1, size=(len(seq),))
    for _ in range(100)
    ],
    configuration=fhe.Configuration(
        enable_unsafe_features=True,
        use_insecure_key_cache=True,
        insecure_key_cache_location=".keys",
        dataflow_parallelize=False, # setting it to True makes the jupyter kernel crash
    ),
    verbose=False,
)

In [33]:
# now we can run our circuit on Seq objects and compare the result with output_seq

# with simulation
fheSim_output_seq = circuit_wrapper(circuit, seq, True)
print('Simulated :', fheSim_output_seq)
assert(np.all(output_seq == fheSim_output_seq))

# and without (slower)


Simulated : [2 3 0 1]
